# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
salaries.head(5)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [74]:
# Your code here:
hourly = len(salaries[salaries['Salary or Hourly']=='Hourly'])
salary = len(salaries[salaries['Salary or Hourly']=='Salary'])
print(hourly)
print(salary)

8022
25161


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:
salaries['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
salary = salaries['Hourly Rate']
mu = salary.mean()
sigma = salary.std()
print(mu)
print(sigma)

32.78855771628023
12.112572684276799


In [8]:
# se obtiene de la tabla z dependiendo el grado de exactitud deseado
z_test = 1.65
x_bar = 95
print(mu)
print(sigma)
n = 10

32.78855771628023
12.112572684276799


In [9]:
standard_error = sigma / np.sqrt(n)
z_stat = (x_bar - mu) / standard_error
z_stat

16.24178935958291

In [10]:
if z_stat < z_test:
    print(f'Valor de z: {z_stat}, menor que {z_test}\n No podemos rechazar la hipotesis')
else:    
    print(f'Valor de z: {z_stat}, mayor que {z_test}\n podemos rechazar la hipotesis, con una confianza del 95%')

Valor de z: 16.24178935958291, mayor que 1.65
 podemos rechazar la hipotesis, con una confianza del 95%


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [11]:
# Your code here:
salary = salaries['Annual Salary']
mu = salary.mean()
sigma = salary.std()
print(mu)
print(sigma)

86786.99979015143
21041.35460162839


In [12]:
# se obtiene de la tabla z dependiendo el grado de exactitud deseado
z_test = 1.65
x_bar = 95
print(mu)
print(sigma)
n = 10

86786.99979015143
21041.35460162839


In [13]:
standard_error = sigma / np.sqrt(n)
z_stat = (x_bar - mu) / standard_error
z_stat

-13.028827251953716

In [14]:
if z_stat < z_test:
    print(f'Valor de z: {z_stat}, menor que {z_test}\n No podemos rechazar la hipotesis')
else:    
    print(f'Valor de z: {z_stat}, mayor que {z_test}\n podemos rechazar la hipotesis, con una confianza del 95%')

Valor de z: -13.028827251953716, menor que 1.65
 No podemos rechazar la hipotesis


Using the `crosstab` function, find the department that has the most hourly workers. 

In [17]:
# Your code here:
salaries['Department'][(salaries['Salary or Hourly']=='Hourly')].value_counts()

STREETS & SAN            1862
WATER MGMNT              1513
OEMC                     1273
AVIATION                 1082
GENERAL SERVICES          765
TRANSPORTN                725
PUBLIC LIBRARY            299
FAMILY & SUPPORT          287
CITY COUNCIL               64
FINANCE                    44
LAW                        40
ANIMAL CONTRL              19
POLICE                     10
MAYOR'S OFFICE              8
CULTURAL AFFAIRS            7
BUSINESS AFFAIRS            7
HUMAN RESOURCES             4
COMMUNITY DEVELOPMENT       4
HEALTH                      3
FIRE                        2
BUDGET & MGMT               2
PROCUREMENT                 2
Name: Department, dtype: int64

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [16]:
# Your code here:
from scipy.stats import ttest_rel, ttest_1samp

In [53]:
hourly_less_35 = salaries['Department'][(salaries['Salary or Hourly']=='Hourly')&(salaries['Hourly Rate']<=35)].value_counts().reset_index()
hourly_bigg_35 = salaries['Department'][(salaries['Salary or Hourly']=='Hourly')&(salaries['Hourly Rate']>35)].value_counts().reset_index()
print(hourly_less_35)
print(hourly_bigg_35)



                    index  Department
0                    OEMC        1229
1           STREETS & SAN         421
2                AVIATION         323
3          PUBLIC LIBRARY         288
4        FAMILY & SUPPORT         287
5        GENERAL SERVICES         134
6             WATER MGMNT         105
7            CITY COUNCIL          64
8                 FINANCE          41
9                     LAW          39
10             TRANSPORTN          25
11          ANIMAL CONTRL          18
12                 POLICE          10
13         MAYOR'S OFFICE           8
14       CULTURAL AFFAIRS           7
15       BUSINESS AFFAIRS           7
16        HUMAN RESOURCES           4
17  COMMUNITY DEVELOPMENT           2
18          BUDGET & MGMT           2
                    index  Department
0           STREETS & SAN        1441
1             WATER MGMNT        1408
2                AVIATION         759
3              TRANSPORTN         700
4        GENERAL SERVICES         631
5           

In [54]:
salary_test = pd.merge(hourly_less_35, hourly_bigg_35, on='index', how='outer')
salary_test

,index,Department_x,Department_y
0,OEMC,1229.0,44.0
1,STREETS & SAN,421.0,1441.0
2,AVIATION,323.0,759.0
3,PUBLIC LIBRARY,288.0,11.0
4,FAMILY & SUPPORT,287.0,NaN
5,GENERAL SERVICES,134.0,631.0
6,WATER MGMNT,105.0,1408.0
7,CITY COUNCIL,64.0,NaN
8,FINANCE,41.0,3.0
9,LAW,39.0,1.0


In [55]:
salary_test.rename(columns={'index':'Department','Department_x':'less_35','Department_y':'bigg_35'}, inplace=True)
salary_test2 = salary_test.dropna()


In [56]:
salary_test2

,Department,less_35,bigg_35
0,OEMC,1229.0,44.0
1,STREETS & SAN,421.0,1441.0
2,AVIATION,323.0,759.0
3,PUBLIC LIBRARY,288.0,11.0
5,GENERAL SERVICES,134.0,631.0
6,WATER MGMNT,105.0,1408.0
8,FINANCE,41.0,3.0
9,LAW,39.0,1.0
10,TRANSPORTN,25.0,700.0
11,ANIMAL CONTRL,18.0,1.0


In [57]:
ttest_rel(salary_test2.less_35, salary_test2.bigg_35)

Ttest_relResult(statistic=-1.0560480421523528, pvalue=0.3157912938579494)

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [67]:
# Your code here:
import numpy as np
import scipy.stats as st
mean_hourly = salaries['Hourly Rate'].dropna()
interval_confidence = st.t.interval(0.95, len(mean_hourly)-1, loc=np.mean(mean_hourly), scale=st.sem(mean_hourly))
interval_confidence

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [69]:
# Your code here:
mean_hourly = salaries['Annual Salary'].dropna()
interval_confidence = st.t.interval(0.95, len(mean_hourly)-1, loc=np.mean(mean_hourly), scale=st.sem(mean_hourly))
interval_confidence

(86526.99656774187, 87047.00301256099)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [73]:
from statsmodels.stats.proportion import proportions_ztest

In [87]:
# Your code here:
# can we assume anything from our sample
significance = 0.25
# our sample - 95% are good
sample_size = hourly
sample_success = hourly/1.05
# our Ho is  95%
null_hypothesis = 0.95
stat, p_value = proportions_ztest(count=sample_success, nobs=sample_size, value=null_hypothesis, alternative='larger')

print(f'z_stat: {stat}, p_value: {p_value}')
print()
if p_value > significance:
   print ("Fail to reject the null hypothesis - we have nothing else to say")
else:
   print ("Reject the null hypothesis - suggest the alternative hypothesis is true")

z_stat: 1.0013740559850708, p_value: 0.15832300103312313

Reject the null hypothesis - suggest the alternative hypothesis is true
